# Testing notebook

In [2]:
import xlsxwriter
import pandas as pd
import numpy as np

from classification_power_predictor import classification_power_predictor
from writing import *

## We need some testing data

In [3]:
# frame size
n = 1000

test_frame = pd.DataFrame({"numeric_variable": np.random.normal(10, 5, n), 
                           "object_variable": np.round(np.random.uniform(0, 10, n)).astype('O'),
                           "column with soo000oo000oo000oo000oo long name": np.random.normal(10, 5, n)})

nv = test_frame["numeric_variable"]
ov = test_frame["object_variable"]

probs1 = nv.apply(lambda x: (x/(max(nv) - min(nv))) + ((np.random.rand())/5))
probs2 = ov.apply(lambda x: (x/(max(ov) - min(ov))) + ((np.random.rand())/5))

f_probs = (probs1 + probs2)/2
f_probs[f_probs >= 1] = 1


Y = np.zeros(n)
Y[f_probs > 0.4] = 1
Y[f_probs > 0.7] = 2

Y = pd.Series(Y)
Y_strs = Y.replace({0: 'cetegory 0', 1:'category 1', 2:'category 2'})
Y.value_counts()
Y_binary = Y.replace({2:1})

na_containts_frame = test_frame.copy()

for col in test_frame.columns:
    na_containts_frame.loc[na_containts_frame.sample(int(n/50)).index, col] = np.NaN

## Computions funcitons

In [4]:
from computions import *

### get_describe_nominal function test

Basic situation

In [5]:
get_describe_nominal(test_frame.iloc[:,1], Y)

object_variable


,count,1.0,1.0%,0.0,0.0%,2.0,2.0%
object_variable,,,,,,,
8.0,121,91.0,75.206612,0.0,0.000000,30.0,24.793388
6.0,112,108.0,96.428571,3.0,2.678571,1.0,0.892857
2.0,105,20.0,19.047619,85.0,80.952381,0.0,0.000000
9.0,102,52.0,50.980392,0.0,0.000000,50.0,49.019608
7.0,99,91.0,91.919192,0.0,0.000000,8.0,8.080808
1.0,97,10.0,10.309278,87.0,89.690722,0.0,0.000000
5.0,95,84.0,88.421053,11.0,11.578947,0.0,0.000000
4.0,87,63.0,72.413793,24.0,27.586207,0.0,0.000000
3.0,86,43.0,50.000000,43.0,50.000000,0.0,0.000000


Data frame with emptys

In [6]:
test = get_describe_nominal(na_containts_frame.iloc[:,1], Y)
print("Is all obs. are in table - ",  str(test['count'].sum() == n))
test = get_describe_nominal(na_containts_frame.fillna('empty').iloc[:,1], Y)
print("Is all obs. are in table - ",  str(test['count'].sum() == n))

object_variable
Is all obs. are in table -  False
object_variable
Is all obs. are in table -  True


### AUC computing test

AUC numeric - simple fucntion

In [7]:
print(get_AUC_numeric(test_frame['numeric_variable'] ,Y))
print(get_AUC_numeric(test_frame['numeric_variable'], Y_strs))
print(get_AUC_numeric(na_containts_frame['numeric_variable'], Y_strs))
print(get_AUC_numeric(test_frame['numeric_variable'], Y_binary))

{1.0: 0.5288623952508241, 0.0: 0.31888483367988407, 2.0: 0.7664069431389946}
{'category 1': 0.5288623952508241, 'cetegory 0': 0.31888483367988407, 'category 2': 0.7664069431389946}
{'category 1': 0.5332787081135522, 'cetegory 0': 0.31555475109482073, 'category 2': 0.7666503586231088}
{1.0: 0.681115166320116, 0.0: 0.31888483367988407}


AUC nominal

In [8]:
non_table = get_AUC_nominal(test_frame['object_variable'], Y)
dn_tab = get_describe_nominal(test_frame.iloc[:,1], Y_strs)
with_table = get_AUC_nominal(test_frame['object_variable'], Y_strs, dn_tab)
print(with_table)

print('AUC bynary')
print(get_AUC_nominal(test_frame['object_variable'], Y_binary))

print('AUC with emptys - needs to be replaces in previous steps')
print(get_AUC_nominal(na_containts_frame['object_variable'].fillna('empty'), Y_strs))

object_variable
object_variable
{'category 1': 0.8638568195608324, 'cetegory 0': 0.9532810841463946, 'category 2': 0.9316359068509035}
AUC bynary
object_variable
{1.0: 0.9532810841463947, 0.0: 0.9532810841463946}
AUC with emptys - needs to be replaces in previous steps
object_variable
{'category 1': 0.8636559307302514, 'cetegory 0': 0.9504529393802801, 'category 2': 0.9289698609439647}


Get publish AUC for any predictor type

In [16]:
non_table = get_full_AUC(test_frame['object_variable'], Y, 'nominal')
print('nominal predictor without table')
print(non_table)

with_table = get_full_AUC(test_frame['object_variable'], Y_strs, 'nominal', dn_tab)
print('moninal predictor with table')
print(with_table)

with_nas = get_full_AUC(na_containts_frame['object_variable'], Y, 'nominal', fillna_nominal = 'empty' )
print('nominal predictor with empty values and fill value')
print(with_nas)


with_nas = get_full_AUC(na_containts_frame['object_variable'], Y, 'nominal')
print('nominal predictor with empty values without fill value')
print(with_nas)

numeric_var = get_full_AUC(test_frame['numeric_variable'], Y, 'numeric')
print('numeric predictor')
print(numeric_var)

object_variable
nominal predictor without table
{1.0: {'AUC': 0.8638568195608324, 'rel_type': 1, 'GINI': 0.7277136391216648}, 0.0: {'AUC': 0.9532810841463946, 'rel_type': 1, 'GINI': 0.9065621682927891}, 2.0: {'AUC': 0.9316359068509035, 'rel_type': 1, 'GINI': 0.863271813701807}}
moninal predictor with table
{'category 1': {'AUC': 0.8638568195608324, 'rel_type': 1, 'GINI': 0.7277136391216648}, 'cetegory 0': {'AUC': 0.9532810841463946, 'rel_type': 1, 'GINI': 0.9065621682927891}, 'category 2': {'AUC': 0.9316359068509035, 'rel_type': 1, 'GINI': 0.863271813701807}}
object_variable
nominal predictor with empty values and fill value
{1.0: {'AUC': 0.8636559307302514, 'rel_type': 1, 'GINI': 0.7273118614605028}, 0.0: {'AUC': 0.9504529393802801, 'rel_type': 1, 'GINI': 0.9009058787605602}, 2.0: {'AUC': 0.9289698609439647, 'rel_type': 1, 'GINI': 0.8579397218879294}}
object_variable
nominal predictor with empty values without fill value
{1.0: {'AUC': 0.8665178590518219, 'rel_type': 1, 'GINI': 0.73303

Decoding AUC info as DataFrame

In [9]:
numeric_var = get_full_AUC(test_frame['numeric_variable'], Y, 'numeric')
AUC_info_to_DataFrame(numeric_var,'numeric_variable')

0.0                          1.0                     \
                       AUC rel_type      GINI       AUC rel_type      GINI   
numeric_variable  0.678353       -1  0.356705  0.575492        1  0.150984   

                       2.0                     
                       AUC rel_type      GINI  
numeric_variable  0.779634        1  0.559267

Getting indicators from computions uotput

In [10]:
comp_result =  get_all_comuptions(test_frame['numeric_variable'], Y)
print('full numeric data')
get_predictor_row(comp_result)
print('numeric data with nas, with str categoryes')
comp_result =  get_all_comuptions(na_containts_frame['numeric_variable'], Y_strs)
get_predictor_row(comp_result)

full numeric data
numeric data with nas, with str categoryes


cetegory 0                    category 1                     \
                        AUC rel_type      GINI        AUC rel_type      GINI   
numeric_variable   0.673652       -1  0.347304     0.5698        1  0.139601   

                 category 2                    Emptys Emptys part  
                        AUC rel_type      GINI                     
numeric_variable   0.784307        1  0.568615     20        0.02

## Writing to excel

Adding a new sheet

In [11]:
xl_writer = pd.ExcelWriter("test_result/sheet_creation.xlsx",engine='xlsxwriter')

for i in range(20):
    add_sheet(xl_writer, 'some name')
    add_sheet(xl_writer, 'some very very very long long name')
xl_writer.close()

Adding a info about different predictors

In [12]:
my_cpp = classification_power_predictor(test_frame, Y)
my_cpp.update_predictors()

object_variable


Let's show final table

In [13]:
my_cpp = classification_power_predictor(test_frame, Y)
my_cpp.update_predictors()
#my_cpp.get_predictors_data()
my_cpp.result_DF

object_variable


0.0                     \
                                                    AUC rel_type      GINI   
numeric_variable                               0.678353       -1  0.356705   
object_variable                                0.950228        1  0.900456   
column with soo000oo000oo000oo000oo long name  0.502247        1  0.004495   

                                                    1.0                     \
                                                    AUC rel_type      GINI   
numeric_variable                               0.575492        1  0.150984   
object_variable                                0.875541        1  0.751082   
column with soo000oo000oo000oo000oo long name  0.506268       -1  0.012536   

                                                    2.0                     \
                                                    AUC rel_type      GINI   
numeric_variable                               0.779634        1  0.559267   
object_variable                                0.930238        1  0.860476   
column with soo000oo000oo000oo000oo long name  0.512613        1  0.025226   

                                              Emptys Emptys part  
                                                                  
numeric_variable                                   0         0.0  
object_variable                                    0         0.0  
column with soo000oo000oo000oo000oo long name      0         0.0

Writig a double header table

In [14]:


xl_writer = pd.ExcelWriter("test_result/double_header_saver.xlsx",engine='xlsxwriter')
ws = xl_writer.book.add_worksheet('test')
xl_writer.sheets['test'] = ws      

my_cpp = classification_power_predictor(test_frame, Y)
my_cpp.update_predictors()
my_cpp.my_writer = xl_writer

print_double_column_header(  my_cpp, ws, 
                            my_cpp.result_DF.loc[['numeric_variable'],:],
                            "A1", 'Hello World', 
                            xl_writer.book.add_format(my_cpp.default_header_format))

xl_writer.close()

object_variable


Writing info about some predictor

In [15]:
xl_writer = pd.ExcelWriter("test_result/writing_a_value.xlsx",engine='xlsxwriter')

ws = xl_writer.book.add_worksheet('test')
xl_writer.sheets['test'] = ws                             

my_cpp = classification_power_predictor(test_frame, Y)
my_cpp.update_predictors()
my_cpp.my_writer = xl_writer

default_predictor_printer(my_cpp, ws, my_cpp.get_predictors_data()['numeric_variable'], 
{'header_format':xl_writer.book.add_format(my_cpp.default_header_format),
  'desc_format':xl_writer.book.add_format(my_cpp.default_header_format),
  'class_ab_format':xl_writer.book.add_format(my_cpp.default_header_format)})

xl_writer.close()

object_variable


In [17]:
xl_writer = pd.ExcelWriter("test_result/result_test.xlsx",engine='xlsxwriter')
my_cpp.write_to_book(xl_writer)
xl_writer.close()